In [1]:
print("hello")

hello


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
# Charger le modèle et le tokenizer
MODEL_NAME = "/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b/1"  # Ajuste le chemin si besoin
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, 
    torch_dtype=torch.float16,  
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [67]:
description="""||Version details||Environment infos||
| * *VERSION* : 6.6.1.4-rc
 * *VERSION_D* : 06-01-2025_13:53:31
 * *VERSION_HASH* : e27cdcaa| * *VM* : visage-apac-2.nx-presales.stratus-neoxam.com ({color:#de350b}*APAC Demo env - Do not change anything on this VM*{color}) - Unable to reproduce it on another env
 * *USER* : loic.brossard|
{panel:title=Steps|borderStyle=dashed|borderColor=#0ed8b8|titleBGColor=#0ed8b8|bgColor=#ffffff}
*1-* Go to Quote aggregate

*2-* On config mode, click on Query filter
 *3-* In the right panel, click on "+" to add a new criteria
{panel}
{panel:title=Current result|borderStyle=dashed|borderColor=#0ed8b8|titleBGColor=#0ed8b8|bgColor=#ffffff}
*3-* Instead of displaying fields from the business/technical dictionary, an "Error:null" message is displayed. In tomcat logs, an NPE is displayed

(logs + screenshot below)
{panel}
{panel:title=Expected result|borderStyle=dashed|borderColor=#0ed8b8|titleBGColor=#0ed8b8|bgColor=#ffffff}
Popup with MxSecurity business/technical fields are displayed 
{panel}
{code:java}
2025-01-16 16:38:39,856 +08:00 ERROR [RunningCmdThread-2809] || client ||java.lang.NullPointerException: null  at visage.components.dictionary.business.field.BusinessField.<init>(SourceFile:38) ~[smartweb.jar:06-01-2025_13:53:31 - SOME DIR]  at visage.commands.BusinessFieldsCmd.cmd(SourceFile:91) ~[smartweb.jar:06-01-2025_13:53:31 - SOME DIR]  at visage.commands.Cmd$RunningCmd.run(SourceFile:246) ~[smartweb.jar:06-01-2025_13:53:31 - SOME DIR]2025-01-16 16:38:39,856 +08:00 TRACE [RunningCmdThread-2809] || client || ends executing command BUSINESS_FIELDS in 2042 ms.2025-01-16 16:38:39,862 +08:00 TRACE [http-nio-8181-exec-3] || client || Serializing message in 02025-01-16 16:38:50,456 +08:00 TRACE [Report-Memory Thread] || client || total=3298 Mb | free=2778 Mb | used=520 Mb2025-01-16 16:39:08,057 +08:00 TRACE [Server ping thread 0] || client || [0, 0, 1]:[total=24576 Mb | free=20164 Mb | used=4412 Mb, ] 
{code}"""

In [83]:
description1 = r"""Nous rencontrons un problème de fuite mémoire sur l'écran synthétique PRICING_DSR_DET_VIEW_SYNTH, lié aux object set.

L'écran synthétique en question permet de synchroniser 6 écrans de query qui travaillent à partir de la classe MxDataSelectionRequest, et permet aux utilisateurs de visualiser des données de pricing, leur permettant de valider les prix sur lesquels des alertes sont remontées.

Nous avons observé des out of memory côté utilisateur (pas côté serveur) liés à l'utilisation de cet écran. 

Nous avons fait une analyse avec eclipse memory analyser, et ce ce que nous avons observé c'est qu'il y a un empilement d'objectset, liés aux écrans sous-jacents de l'écran synthétique, avec des références sur plein de versions différentes d'objets MxDataSelectionRequest.

*************************************************

*Run Update :* 

l'erreur OutOfMemory est reproduite sur l'environement interne : [http://nxdh-lodh-idi-new:8080/nxdh/] user SA 

deux  hprof sont générés et sont disponibles sur 

\\bams-svgfile01\Globalia\02_Projet_Support_DH\EMEA\LODH\NXDH-28408

le client n'envisage pas d'augmenter la mémoire pour les jvm client : c'était juste une solution temporaire 

{color:#de350b}*Merci d'analyser le hprof fournis et d'identifier la rootcause de cet OOM.*{color}"""


In [84]:
prompt = f"""
You are an incident log analysis assistant.  
Your task is to extract and structure the requested information **without modifying the output format**.  
You must understand both **French and English**.  

Strictly follow the structure below:

---

📂 **Full Logs:**  
{{{{If logs are present, extract them while preserving their original format.}}}}  
{{{{If no logs are detected, write: "No logs detected in the description."}}}}  
{{{{If logs are stored in a file, image, or video, write:  
"The logs are stored in a file/image/video and are not directly included in the description."}}}}  

📝 **Explanatory Text:**  
(Extract only the contextual text that describes the issue, the steps to reproduce it, and the expected result.)  

⚠️ **Error Analysis:**  
- 🔥 **Error Type**: (e.g., NullPointerException, SQL Exception, etc.)  
- 📝 **Error Message**: (main error message)  
- 📌 **File and Line**: (e.g., `BusinessField.<init>(SourceFile:38)`)  
- 🕒 **Timestamp**: (log timestamp, e.g., `2025-01-16 16:38:39,856 +08:00`)  
- ⚙️ **Affected Thread**: (name of the thread executing the command)  
- 📂 **Impacted Module or Component**: (e.g., `BusinessFieldsCmd`)  

---

⚠️ **IMPORTANT**:  
- **Never modify this format.**  
- **If any information is missing, write "Not available" instead of removing it.**  
- **Respect indentation, punctuation, and line breaks.**  
- **Do not add extra sections or explanations.**  

🔍 Here is the full description:  
{description1}

---

⚠️ **IMPORTANT:**  
- The response must be **in JSON format only**.  
- Never include the prompt text in the response.  
- Return only a structured JSON object with no additional text.  
- The JSON output must follow this exact structure follow only the structure its just an exemple you have to fill this with the necessary inforamtion:  

```json
{{
  "Full Logs": 
  "Explanatory Text": 
  "Error Analysis": {{
    "Error Type": 
    "Error Message": 
    "File and Line":
    "Timestamp": 
    "Affected Thread":
    "Impacted Module or Component": 
  }}
}}
"""

In [85]:
# Tokenizer l'entrée et générer une réponse
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
output = model.generate(**inputs, max_length=10000)
response = tokenizer.decode(output[0], skip_special_tokens=True)

# Afficher la sortie
print(response)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



You are an incident log analysis assistant.  
Your task is to extract and structure the requested information **without modifying the output format**.  
You must understand both **French and English**.  

Strictly follow the structure below:

---

📂 **Full Logs:**  
{{If logs are present, extract them while preserving their original format.}}  
{{If no logs are detected, write: "No logs detected in the description."}}  
{{If logs are stored in a file, image, or video, write:  
"The logs are stored in a file/image/video and are not directly included in the description."}}  

📝 **Explanatory Text:**  
(Extract only the contextual text that describes the issue, the steps to reproduce it, and the expected result.)  

⚠️ **Error Analysis:**  
- 🔥 **Error Type**: (e.g., NullPointerException, SQL Exception, etc.)  
- 📝 **Error Message**: (main error message)  
- 📌 **File and Line**: (e.g., `BusinessField.<init>(SourceFile:38)`)  
- 🕒 **Timestamp**: (log timestamp, e.g., `2025-01-16 16:38:39,